In [26]:
import pandas as pd
import srt, os
from datetime import timedelta

In [67]:
def create_srt(df, video_name, video_dir):
    temp_df = df[df["file_name"] == video_name].copy()

    temp_df.reset_index(drop=True, inplace=True)

    temp_df = (
        temp_df[["segment_id", "speaker", "word", "word_start", "word_end"]]
        .groupby(["segment_id", "speaker"], as_index=False)
        .agg({"word": " ".join, "word_start": min, "word_end": max})
    )

    temp_df["segment_id"] = temp_df.index + 1

    srt_list = temp_df.apply(
        lambda row: srt.Subtitle(
            index=row["segment_id"],
            start=timedelta(seconds=row["word_start"] if row["word_start"] < 0.1 else row["word_start"] - 0.1),
            end=timedelta(seconds=row["word_end"] + 0.1),
            content=f"{row['speaker']}: {row['word']}",
        ),
        axis=1,
    ).to_list()

    srt_string = srt.compose(srt_list)
    with open(
        os.path.join(
            video_dir,
            os.path.splitext(video_name)[0] + ".srt",
        ),
        "w",
    ) as f:
        f.write(srt_string)


In [75]:
df = pd.read_excel("../../Dataset/Transcriptions_MediumModel.xlsx", sheet_name="WordLevel")

In [76]:
file_list = df[df["speaker"].notnull()]["file_name"].unique()
for item in file_list:
    create_srt(df, item, "../../Dataset/Videos")